In [1]:
import pandas as pd

In [2]:
df1 = pd.read_excel('범죄와의전쟁.xlsx',index_col=0)
df2 = pd.read_excel('광해.xlsx',index_col=0)
df3 = pd.read_excel('내부자들.xlsx',index_col=0)
df4 = pd.read_excel('더킹.xlsx',index_col=0)

In [3]:
# nan값 있는 항 모두제거
df1 = df1.dropna(axis=0)
df2 = df2.dropna(axis=0)
df3 = df3.dropna(axis=0)
df4 = df4.dropna(axis=0)

In [4]:
with open('nsmc1.txt', 'w', encoding='utf8') as f:  
    f.write('\n'.join(df1['review']))  

In [5]:
from sentencepiece import SentencePieceTrainer
SentencePieceTrainer.Train('--input=nsmc1.txt --model_prefix=nsmc1 --vocab_size=3000') 

True

In [6]:
from sentencepiece import SentencePieceProcessor
sp1 = SentencePieceProcessor()
sp1.Load("nsmc1.model")

True

In [7]:
from sklearn.feature_extraction.text import CountVectorizer 

In [8]:
cv1 = CountVectorizer(lowercase=False, tokenizer=sp1.encode_as_pieces)

In [9]:
tdm1 = cv1.fit_transform(df1['review'])

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
x1 = tdm1
y1 = df1['sentiment']

In [12]:
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.2, random_state=42)

In [13]:
import joblib

In [14]:
cv1.tokenizer = None

In [15]:
joblib.dump((cv1,x1_train,x1_test,y1_train,y1_test), 'nsmc1.pkl')

['nsmc1.pkl']

In [16]:
import tensorflow as tf

In [17]:
model1 = tf.keras.models.Sequential()

In [18]:
x1_train.shape

(1463, 3038)

In [19]:
model1.add(
    tf.keras.layers.Dense(
        1, 
        input_shape=(3038,),
        activation='sigmoid', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001) 
    ))  

In [20]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 3039      
Total params: 3,039
Trainable params: 3,039
Non-trainable params: 0
_________________________________________________________________


In [21]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

In [22]:
model1.fit(x1_train.toarray(), y1_train, epochs=100, validation_split=0.1,
         callbacks=[tf.keras.callbacks.EarlyStopping()])

Train on 1316 samples, validate on 147 samples
Epoch 1/100
1316/1316 [==============================] - 0s 232us/sample - loss: 0.6142 - accuracy: 0.7758 - val_loss: 0.4971 - val_accuracy: 0.9456
Epoch 2/100
1316/1316 [==============================] - 0s 39us/sample - loss: 0.5161 - accuracy: 0.8708 - val_loss: 0.4260 - val_accuracy: 0.9456
Epoch 3/100
1316/1316 [==============================] - 0s 36us/sample - loss: 0.4757 - accuracy: 0.8701 - val_loss: 0.3949 - val_accuracy: 0.9456
Epoch 4/100
1316/1316 [==============================] - 0s 35us/sample - loss: 0.4491 - accuracy: 0.8701 - val_loss: 0.3731 - val_accuracy: 0.9456
Epoch 5/100
1316/1316 [==============================] - 0s 36us/sample - loss: 0.4271 - accuracy: 0.8708 - val_loss: 0.3605 - val_accuracy: 0.9456
Epoch 6/100
1316/1316 [==============================] - 0s 35us/sample - loss: 0.4081 - accuracy: 0.8708 - val_loss: 0.3512 - val_accuracy: 0.9456
Epoch 7/100
1316/1316 [==============================] - 0s 36us

In [23]:
model1.evaluate(x1_test.toarray(), y1_test)

366/366 [==============================] - 0s 44us/sample - loss: 0.3559 - accuracy: 0.9071


[0.3558588099609959, 0.90710384]

In [24]:
weights1,_ = model1.trainable_weights

In [25]:
token_weight1 = pd.DataFrame({'토큰': cv1.get_feature_names(), '가중치': weights1.numpy().flat})

In [26]:
token_weight1.sort_values('가중치').head()

,토큰,가중치
73,T,-0.318324
2609,조선,-0.303892
634,▁별로,-0.268537
74,V,-0.267715
1139,▁재미없,-0.255375


In [27]:
token_weight1.sort_values('가중치').tail()

,토큰,가중치
1294,▁최고,0.375433
950,▁연기,0.379698
1151,▁재밌게,0.381203
1,!!,0.392650
738,▁살아있네,0.440150


In [28]:
with open('nsmc2.txt', 'w', encoding='utf8') as f:  
    f.write('\n'.join(df2['review']))  

SentencePieceTrainer.Train('--input=nsmc2.txt --model_prefix=nsmc2 --vocab_size=3000')  # 총 어휘수 3000개

sp2 = SentencePieceProcessor()
sp2.Load("nsmc2.model")


cv2 = CountVectorizer(lowercase=False, tokenizer=sp2.encode_as_pieces)

tdm2 = cv2.fit_transform(df2['review'])

x2 = tdm2
y2 = df2['sentiment']

x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, test_size=0.2, random_state=42)

cv2.tokenizer = None

joblib.dump((cv2,x2_train,x2_test,y2_train,y2_test), 'nsmc2.pkl')

model2 = tf.keras.models.Sequential()

model2.add(
    tf.keras.layers.Dense(
        1, 
        input_shape=(3077,),
        activation='sigmoid', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001) 
    ))  

In [29]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 3078      
Total params: 3,078
Trainable params: 3,078
Non-trainable params: 0
_________________________________________________________________


In [30]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

model2.fit(x2_train.toarray(), y2_train, epochs=100, validation_split=0.1,
         callbacks=[tf.keras.callbacks.EarlyStopping()])

Train on 2466 samples, validate on 275 samples
Epoch 1/100
2466/2466 [==============================] - 0s 116us/sample - loss: 0.5609 - accuracy: 0.8447 - val_loss: 0.4602 - val_accuracy: 0.9164
Epoch 2/100
2466/2466 [==============================] - 0s 35us/sample - loss: 0.4357 - accuracy: 0.9075 - val_loss: 0.4054 - val_accuracy: 0.9164
Epoch 3/100
2466/2466 [==============================] - 0s 36us/sample - loss: 0.3917 - accuracy: 0.9075 - val_loss: 0.3779 - val_accuracy: 0.9164
Epoch 4/100
2466/2466 [==============================] - 0s 36us/sample - loss: 0.3626 - accuracy: 0.9079 - val_loss: 0.3599 - val_accuracy: 0.9164
Epoch 5/100
2466/2466 [==============================] - 0s 34us/sample - loss: 0.3389 - accuracy: 0.9084 - val_loss: 0.3453 - val_accuracy: 0.9164
Epoch 6/100
2466/2466 [==============================] - 0s 35us/sample - loss: 0.3188 - accuracy: 0.9092 - val_loss: 0.3334 - val_accuracy: 0.9164
Epoch 7/100
2466/2466 [==============================] - 0s 34us

In [31]:
model2.evaluate(x2_test.toarray(), y2_test)

686/686 [==============================] - 0s 25us/sample - loss: 0.2832 - accuracy: 0.9155


[0.28324324335718293, 0.9154519]

In [32]:
weights2,_ = model2.trainable_weights

token_weight2 = pd.DataFrame({'토큰': cv2.get_feature_names(), '가중치': weights2.numpy().flat})

token_weight2.sort_values('가중치').head()

,토큰,가중치
1694,냐,-0.508830
443,▁데이브,-0.464805
1049,▁이게,-0.447141
1377,▁표절,-0.355221
854,▁알바들,-0.348203


In [33]:
token_weight2.sort_values('가중치').tail()

,토큰,가중치
1066,▁이병헌의,0.531894
1329,▁최고,0.531994
0,!,0.567077
1129,▁잘,0.578303
915,▁연기,0.591277


In [34]:
with open('nsmc3.txt', 'w', encoding='utf8') as f:  
    f.write('\n'.join(df3['review']))  

SentencePieceTrainer.Train('--input=nsmc3.txt --model_prefix=nsmc3 --vocab_size=3000')  # 총 어휘수 3000개

sp3 = SentencePieceProcessor()
sp3.Load("nsmc3.model")


cv3 = CountVectorizer(lowercase=False, tokenizer=sp3.encode_as_pieces)

tdm3 = cv3.fit_transform(df3['review'])

x3 = tdm3
y3 = df3['sentiment']

x3_train, x3_test, y3_train, y3_test = train_test_split(x3, y3, test_size=0.2, random_state=43)

cv3.tokenizer = None

joblib.dump((cv3,x3_train,x3_test,y3_train,y3_test), 'nsmc3.pkl')

model3 = tf.keras.models.Sequential()

model3.add(
    tf.keras.layers.Dense(
        1, 
        input_shape=(2976,),
        activation='sigmoid', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001)   
    ))  

In [35]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 2977      
Total params: 2,977
Trainable params: 2,977
Non-trainable params: 0
_________________________________________________________________


In [36]:
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

model3.fit(x3_train.toarray(), y3_train, epochs=100, validation_split=0.1,
         callbacks=[tf.keras.callbacks.EarlyStopping()])

Train on 709 samples, validate on 79 samples
Epoch 1/100
709/709 [==============================] - 0s 307us/sample - loss: 0.6214 - accuracy: 0.7983 - val_loss: 0.5621 - val_accuracy: 0.9114
Epoch 2/100
709/709 [==============================] - 0s 54us/sample - loss: 0.5120 - accuracy: 0.9323 - val_loss: 0.4987 - val_accuracy: 0.9494
Epoch 3/100
709/709 [==============================] - 0s 40us/sample - loss: 0.4494 - accuracy: 0.9394 - val_loss: 0.4621 - val_accuracy: 0.9494
Epoch 4/100
709/709 [==============================] - 0s 40us/sample - loss: 0.4112 - accuracy: 0.9394 - val_loss: 0.4375 - val_accuracy: 0.9494
Epoch 5/100
709/709 [==============================] - 0s 41us/sample - loss: 0.3833 - accuracy: 0.9394 - val_loss: 0.4207 - val_accuracy: 0.9494
Epoch 6/100
709/709 [==============================] - 0s 38us/sample - loss: 0.3625 - accuracy: 0.9394 - val_loss: 0.4083 - val_accuracy: 0.9494
Epoch 7/100
709/709 [==============================] - 0s 37us/sample - loss: 

709/709 [==============================] - 0s 37us/sample - loss: 0.1553 - accuracy: 0.9690 - val_loss: 0.2784 - val_accuracy: 0.9494
Epoch 57/100
709/709 [==============================] - 0s 38us/sample - loss: 0.1545 - accuracy: 0.9690 - val_loss: 0.2778 - val_accuracy: 0.9494
Epoch 58/100
709/709 [==============================] - 0s 37us/sample - loss: 0.1534 - accuracy: 0.9690 - val_loss: 0.2766 - val_accuracy: 0.9494
Epoch 59/100
709/709 [==============================] - 0s 37us/sample - loss: 0.1525 - accuracy: 0.9690 - val_loss: 0.2756 - val_accuracy: 0.9494
Epoch 60/100
709/709 [==============================] - 0s 38us/sample - loss: 0.1517 - accuracy: 0.9704 - val_loss: 0.2744 - val_accuracy: 0.9494
Epoch 61/100
709/709 [==============================] - 0s 37us/sample - loss: 0.1508 - accuracy: 0.9690 - val_loss: 0.2737 - val_accuracy: 0.9494
Epoch 62/100
709/709 [==============================] - 0s 37us/sample - loss: 0.1499 - accuracy: 0.9704 - val_loss: 0.2729 - val_a

In [37]:
model3.evaluate(x3_test.toarray(), y3_test)

197/197 [==============================] - 0s 36us/sample - loss: 0.1884 - accuracy: 0.9695


[0.1883635123822895, 0.96954316]

In [38]:
weights3,_ = model3.trainable_weights

token_weight3 = pd.DataFrame({'토큰': cv3.get_feature_names(), '가중치': weights3.numpy().flat})

token_weight3.sort_values('가중치').head()

,토큰,가중치
577,▁무슨,-0.490435
1783,내용인지,-0.473362
1210,▁재미없,-0.427505
2900,해서,-0.406046
398,▁대,-0.381082


In [39]:
token_weight3.sort_values('가중치').tail()

,토큰,가중치
90,~~,0.543954
134,▁3,0.551459
0,!,0.576938
1392,▁최고,0.729540
248,▁굿,0.749861


In [40]:
with open('nsmc4.txt', 'w', encoding='utf8') as f:  
    f.write('\n'.join(df4['review']))  

SentencePieceTrainer.Train('--input=nsmc4.txt --model_prefix=nsmc4 --vocab_size=3000')  # 총 어휘수 3000개

sp4 = SentencePieceProcessor()
sp4.Load("nsmc4.model")


cv4 = CountVectorizer(lowercase=False, tokenizer=sp4.encode_as_pieces)

tdm4 = cv4.fit_transform(df4['review'])

x4 = tdm4
y4 = df4['sentiment']

x4_train, x4_test, y4_train, y4_test = train_test_split(x4, y4, test_size=0.2, random_state=44)

cv4.tokenizer = None

joblib.dump((cv4,x4_train,x4_test,y4_train,y4_test), 'nsmc4.pkl')

model4 = tf.keras.models.Sequential()

x4_train.shape

model4.add(
    tf.keras.layers.Dense(
        1, 
        input_shape=(3045,),
        activation='sigmoid', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001)  
    ))  

In [41]:
model4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1)                 3046      
Total params: 3,046
Trainable params: 3,046
Non-trainable params: 0
_________________________________________________________________


In [42]:
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

model4.fit(x4_train.toarray(), y4_train, epochs=100, validation_split=0.1,
         callbacks=[tf.keras.callbacks.EarlyStopping()])

Train on 1833 samples, validate on 204 samples
Epoch 1/100
1833/1833 [==============================] - 0s 141us/sample - loss: 0.6286 - accuracy: 0.7943 - val_loss: 0.6106 - val_accuracy: 0.8235
Epoch 2/100
1833/1833 [==============================] - 0s 32us/sample - loss: 0.5646 - accuracy: 0.8358 - val_loss: 0.5866 - val_accuracy: 0.8235
Epoch 3/100
1833/1833 [==============================] - 0s 32us/sample - loss: 0.5267 - accuracy: 0.8347 - val_loss: 0.5690 - val_accuracy: 0.8235
Epoch 4/100
1833/1833 [==============================] - 0s 34us/sample - loss: 0.4954 - accuracy: 0.8352 - val_loss: 0.5533 - val_accuracy: 0.8235
Epoch 5/100
1833/1833 [==============================] - 0s 32us/sample - loss: 0.4678 - accuracy: 0.8363 - val_loss: 0.5394 - val_accuracy: 0.8235
Epoch 6/100
1833/1833 [==============================] - 0s 34us/sample - loss: 0.4440 - accuracy: 0.8396 - val_loss: 0.5266 - val_accuracy: 0.8235
Epoch 7/100
1833/1833 [==============================] - 0s 32us

In [43]:
model4.evaluate(x4_test.toarray(), y4_test)

510/510 [==============================] - 0s 29us/sample - loss: 0.4031 - accuracy: 0.8490


[0.40314812905648173, 0.8490196]

In [44]:
weights4,_ = model4.trainable_weights

token_weight4 = pd.DataFrame({'토큰': cv4.get_feature_names(), '가중치': weights4.numpy().flat})

token_weight4.sort_values('가중치').head()

,토큰,가중치
1365,▁지루하고,-0.438611
606,▁못,-0.427968
1404,▁최악의,-0.391649
870,▁실망,-0.362776
367,▁나레이션,-0.347984


In [45]:
token_weight4.sort_values('가중치').tail()

,토큰,가중치
1202,▁잘,0.505837
1507,▁현실,0.507013
69,^^,0.543683
1243,▁재밌게,0.562631
355,▁꼭,0.597373


- 네이버 에서 best 댓글 10개씩을 추출하여 새로운 데이터로 적용시켜봄.

In [46]:
import requests
import lxml.html

In [47]:
res1 = requests.get('https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=82540&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false')

In [48]:
root1 = lxml.html.fromstring(res1.text)

In [49]:
new_data1 = []
for review in root1.cssselect('.score_reple p'):
    text = review.text_content().strip()
    new_data1.append(text)

In [50]:
cv1.tokenizer = sp1.encode_as_pieces

In [51]:
x1_new = cv1.transform(new_data1) 

In [52]:
model1.predict(x1_new.toarray())

array([[0.95040005],
       [0.70022285],
       [0.7016349 ],
       [0.67459166],
       [0.9607522 ],
       [0.84816766],
       [0.76829845],
       [0.74512774],
       [0.8576355 ],
       [0.73191404]], dtype=float32)

- 상위 10개의 베스트 댓글 분석 결과 대부분이 긍정 반응으로 나타남.

In [53]:
res2 = requests.get('https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=83893&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false')
root2 = lxml.html.fromstring(res2.text)
    
new_data2 = []
for review in root2.cssselect('.score_reple p'):
    text = review.text_content().strip()
    new_data2.append(text)

cv2.tokenizer = sp2.encode_as_pieces
x2_new = cv2.transform(new_data2)
model2.predict(x2_new.toarray())

array([[0.8044546 ],
       [0.96871567],
       [0.92740744],
       [0.9713144 ],
       [0.9750368 ],
       [0.9583796 ],
       [0.8792013 ],
       [0.79396695],
       [0.9498907 ],
       [0.8677815 ]], dtype=float32)

- 상위 10개의 베스트 댓글 분석 결과 대부분이 긍정 반응으로 나타남.

In [54]:
res3 = requests.get('https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=121788&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false')
root3 = lxml.html.fromstring(res3.text)
    
new_data3 = []
for review in root3.cssselect('.score_reple p'):
    text = review.text_content().strip()
    new_data3.append(text)

cv3.tokenizer = sp3.encode_as_pieces
x3_new = cv3.transform(new_data3)
model3.predict(x3_new.toarray())

array([[0.86940473],
       [0.9292755 ],
       [0.9042327 ],
       [0.993309  ],
       [0.8398019 ],
       [0.8648984 ],
       [0.9436459 ],
       [0.9948738 ],
       [0.9672263 ],
       [0.9522758 ]], dtype=float32)

- 상위 10개의 베스트 댓글 분석 결과 대부분이 긍정 반응으로 나타남.

In [55]:
res4 = requests.get('https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=144314&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false')
root4 = lxml.html.fromstring(res4.text)
    
new_data4 = []
for review in root4.cssselect('.score_reple p'):
    text = review.text_content().strip()
    new_data4.append(text)

cv4.tokenizer = sp4.encode_as_pieces
x4_new = cv4.transform(new_data4)
model4.predict(x4_new.toarray())

array([[0.6073537],
       [0.8728601],
       [0.8398966],
       [0.9098527],
       [0.878911 ],
       [0.9556842],
       [0.8199466],
       [0.9635894],
       [0.9390126],
       [0.9537958]], dtype=float32)

- 상위 10개의 베스트 댓글 분석 결과 대부분이 긍정 반응으로 나타남.